In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
from utils import CustomDataset, train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# from pathlib import Path
# sources_path = Path("/content/drive/MyDrive/Data")
# X_train = np.load(str(sources_path / 'X_train.npy'))
# y_train = np.load(str(sources_path / 'Data/y_train.npy'))
# X_test = np.load(str(sources_path / 'Data/X_test.npy'))
# y_test = np.load(str(sources_path / 'Data/y_test.npy'))

In [2]:
X_train = np.load('Data/rnn_X_train.npy')
y_train = np.load('Data/y_train.npy')

X_test = np.load('Data/rnn_X_test.npy')
y_test = np.load('Data/y_test.npy')

voc_size = 5412  # "Data_preparation_RNN.ipynb"
padding_idx = 5411

In [3]:
train_df = CustomDataset(X_train, y_train)
test_df = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_df, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_df, batch_size=1024, shuffle=False)

In [4]:
X, y = next(iter(train_loader))
X.shape

torch.Size([1024, 100])

In [9]:
class RNN(nn.Module):
    def __init__(self, voc_size, embedding_size, padding_idx, hidden_state_size, n_rec_layers, output_size):
        super().__init__()
        self.n_rec_layers = n_rec_layers
        self.hidden_state_size = hidden_state_size

        self.embedding = nn.Embedding(voc_size, embedding_size, padding_idx)
        self.rnn = nn.GRU(embedding_size, hidden_state_size, n_rec_layers, batch_first=True)
        self.fc = nn.Linear(hidden_state_size, output_size)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, X):
        h_0 = torch.zeros(self.n_rec_layers, X.size(0), self.hidden_state_size).to(device)

        X = self.embedding(X)
        X, _ = self.rnn(X, h_0)
        X = X[:, -1, :]
        X = self.fc(X)
        X = self.logsoftmax(X)

        return X

In [ ]:
rnn = RNN(
    voc_size=voc_size,
    embedding_size=512,
    padding_idx=padding_idx,
    hidden_state_size=512,
    n_rec_layers=1,
    output_size=5
).to(device)

optimizer = Adam(rnn.parameters(), lr=1e-4)
train_losses, test_losses, train_f1_scores, test_f1_scores = train(rnn, optimizer, 45, train_loader, test_loader)

  0%|          | 0/45 [00:00<?, ?it/s]

In [7]:
a = torch.IntTensor([[1, 4, 7, 0, 0, 99],
                     [1, 2, 5, 2, 3, 6]]).to(torch.int64)

emb = torch.nn.Embedding(100, 1, padding_idx=99)
emb(a)


tensor([[[-0.5086],
         [ 1.3157],
         [ 0.7384],
         [-0.1474],
         [-0.1474],
         [ 0.0000]],

        [[-0.5086],
         [ 0.7876],
         [ 0.2688],
         [ 0.7876],
         [-2.0134],
         [ 0.9981]]], grad_fn=<EmbeddingBackward0>)